# Homework

Read about difference between GPT-3.5 and GPT-4.

Read about metrics for generarive NLP.

**Advanced**: Generative models are usually very big. Read about model quantization. That may help with inference of big models such as GPT.

**Theory** (5 points): Google form questions.

**Practical task** (10 points): 
1. Choose one:
    * Finetune transformer model for summarization on https://huggingface.co/datasets/samsum.
    * Finetune transformer model for translation on dataset of your choice.
2. Experiment with different prompts.
2. Based on a task you choose, choose a few metrics that are used in generative NLP (BLEU, ROUGE etc), test your finetune models using them, describe their pros and cons relative to the generations your model makes.

3. If you want, you can try use LoRA or prefix tuning for finetuning the model.

## Imports

In [14]:
%pip install py7zr evaluate rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import evaluate
import numpy as np

## Data preparation

In [16]:
samsum = load_dataset("samsum")

In [17]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [55]:
def preprocess_function(examples, prefix):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

T5 model was pre-trained on different tasks, including summarization. During the pre-training stage the prefix `"summarize: "` was used for summarization task, and thus it should be used for summarization inferences. We will try to investigate, what whould happen if we replace this prompt with more precise `"summarize the following dialogue: "` during the fine-tuning on the dataset of dialogues.

In [52]:
tokenized_samsum = samsum.map(preprocess_function, batched=True, fn_kwargs={"prefix": "summarize: "})
tokenized_samsum_new_prompt = samsum.map(preprocess_function, batched=True, fn_kwargs={"prefix": "summarize the following dialogue: "})

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Training

In [10]:
rouge = evaluate.load("rouge")

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/t5-samsum",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_strategy='epoch',
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_samsum["train"],
    eval_dataset=tokenized_samsum["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api") 

wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
trainer.train()
wandb.finish()

wandb: Currently logged in as: yevhenii-azarov. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231106_022236-ucffq3jv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run leafy-tree-10
wandb: ⭐️ View project at https://wandb.ai/yevhenii-azarov/huggingface
wandb: 🚀 View run at https://wandb.ai/yevhenii-azarov/huggingface/runs/ucffq3jv
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.257200,1.873949,0.396900,0.170600,0.332400,0.332000,16.365500
2,2.034600,1.826320,0.404500,0.177800,0.339200,0.338900,16.509800
3,1.985800,1.803390,0.409200,0.182200,0.344200,0.344300,16.467000
4,1.958500,1.786948,0.414500,0.183400,0.347800,0.347900,16.577000
5,1.945600,1.782817,0.414300,0.185000,0.348200,0.348100,16.613700
6,1.931000,1.780898,0.415800,0.185800,0.349200,0.349100,16.636900


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                   eval/gen_len ▁▅▄▆▇█
wandb:                      eval/loss █▄▃▁▁▁
wandb:                    eval/rouge1 ▁▄▆█▇█
wandb:                    eval/rouge2 ▁▄▆▇██
wandb:                    eval/rougeL ▁▄▆▇██
wandb:                 eval/rougeLsum ▁▄▆███
wandb:                   eval/runtime █▁▁▁▂▂
wandb:        eval/samples_per_second ▁██▇▇▇
wandb:          eval/steps_per_second ▁██▇▇▇
wandb:                    train/epoch ▁▂▂▂▃▃▄▄▄▅▅▆▇▇▇███
wandb:              train/global_step ▁▂▂▂▃▃▄▄▄▅▅▆▇▇▇███
wandb:            train/learning_rate █▇▇▆▅▄▄▃▂▂▁
wandb:                     train/loss █▅▃▃▂▂▂▁▁▁▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                   eval/gen_len 16.6369
wandb:                  

In [ ]:
training_args.output_dir = "/kaggle/working/t5-samsum-newprompt"
trainer_new_prompt = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_samsum_new_prompt["train"],
    eval_dataset=tokenized_samsum_new_prompt["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer_new_prompt.train()
wandb.finish()

wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231106_025303-lge5xhsq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run /kaggle/working/t5-samsum
wandb: ⭐️ View project at https://wandb.ai/yevhenii-azarov/huggingface
wandb: 🚀 View run at https://wandb.ai/yevhenii-azarov/huggingface/runs/lge5xhsq


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.921200,1.761207,0.421900,0.192700,0.355300,0.355300,16.647900
2,1.885600,1.746259,0.422800,0.194800,0.357800,0.357700,16.698000
3,1.870700,1.739120,0.424300,0.197900,0.361600,0.361600,16.526900
4,1.858900,1.730590,0.426400,0.199700,0.361500,0.361700,16.627100
5,1.849500,1.728655,0.428100,0.200300,0.363200,0.363300,16.625900
6,1.841300,1.728808,0.428900,0.200700,0.363400,0.363400,16.701700


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                   eval/gen_len ▆█▁▅▅█
wandb:                      eval/loss █▅▃▁▁▁
wandb:                    eval/rouge1 ▁▂▃▆▇█
wandb:                    eval/rouge2 ▁▃▆▇██
wandb:                    eval/rougeL ▁▃▆▆██
wandb:                 eval/rougeLsum ▁▃▆▇██
wandb:                   eval/runtime █▆▃▁▂▃
wandb:        eval/samples_per_second ▁▃▆█▇▆
wandb:          eval/steps_per_second ▁▃▆█▇▆
wandb:                    train/epoch ▁▂▂▂▃▃▄▄▄▅▅▆▇▇▇███
wandb:              train/global_step ▁▂▂▂▃▃▄▄▄▅▅▆▇▇▇███
wandb:            train/learning_rate █▇▇▆▅▅▄▃▂▂▁
wandb:                     train/loss ██▅▅▄▃▃▂▂▁▂
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                   eval/gen_len 16.7017
wandb:                  

## Evaluation

Since there is no significant signs of overfitting, we will use the last snapshots for both cases for evaluation.

ROUGE score (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics commonly used for text summarization tasks. ROUGE was designed to evaluate the quality of machine-generated summaries by comparing them to reference summaries provided by humans. Here we will compare several different variants of the ROUGE scores.

**ROUGE-N**

Measures overlap of n-grams (contiguous sequences of n words) between the reference and generated summaries.
- ROUGE-1: variant with unigrams
- ROUGE-2: variant with bigrams

ROUGE-N is sensitive to phrase matching. It is simple and easy to compute. Among cons, it does not explicitly account for word order, which may limit its ability to capture the coherence and fluency of summaries. Also, it may penalize synonyms or paraphrased expressions that convey similar meanings but use different words.

**ROUGE-L**

Computes the longest common subsequence between reference and generated summaries, considering word sequences.
- ROUGE-L: computes average recall based among sentences (splits text by '.')
- ROUGE-Lsum: computes average recall among lines (splits text by '\n') 

Longest common substring consideration can be beneficial for evaluating the coherence and structure of summaries. ROUGE-L is less sensitive to variations in word choice and allows for partial matches, making it more forgiving in certain cases. From the drawbacks perspective, the longest common subsequence approach might not always reflect the semantic similarity, especially in cases where there are multiple ways to form a valid subsequence.

In [ ]:
!pip install py7zr evaluate rouge_score

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
samsum_test = load_dataset('samsum', split='test')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


In [ ]:
def add_prefix(examples):
    examples['dialogue_prompt'] = ["summarize: " + doc for doc in examples['dialogue']]
    examples['dialogue_prompt_new'] = ["summarize the following dialogue: " + doc for doc in examples['dialogue']]
    return examples

samsum_test = samsum_test.map(add_prefix, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
checkpoint_base = 't5-small'
checkpoint_1 = '/kaggle/input/t5-samsum-tuning/t5-samsum/checkpoint-5526'
checkpoint_2 = '/kaggle/input/t5-samsum-tuning/t5-samsum-newprompt/checkpoint-5526'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint_base)
model_base = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_base)
model_1 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_1)
model_2 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_2)

In [ ]:
pipeline_base = pipeline("summarization", model_base, tokenizer=tokenizer, max_length=300, device=0)
pipeline_1 = pipeline("summarization", model_1, tokenizer=tokenizer, max_length=300, device=0)
pipeline_2 = pipeline("summarization", model_2, tokenizer=tokenizer, max_length=300, device=0)

In [ ]:
print(samsum_test['dialogue'][5])

Benjamin: Hey guys, what are we doing with the keys today?
Hilary: I've got them. Whoever wants them can meet me at lunchtime or after
Elliot: I'm ok. We're meeting for the drinks in the evening anyway and I guess we'll be going back to the apartment together?
Hilary: Yeah, I guess so
Daniel: I'm with Hilary atm and won't let go of her for the rest of the day, so any option you guys choose is good for me
Benjamin: Hmm I might actually pass by at lunchtime, take the keys and go take a nap. I'm sooo tired after yesterday
Hilary: Sounds good. We'll be having lunch with some French people (the ones who work on the history of food in colonial Mexico - I already see you yawning your head off)
Benjamin: YAAAAWN 🙊 Where and where are you meeting?
Hilary: So I'm meeting them at the entrance to the conference hall at 2 pm and then we'll head to this place called La Cantina. Italian cuisine, which is quite funny, but that's what they've chosen
Benjamin: Interesting 😱 To be honest, Hilary, I almos

In [ ]:
print(pipeline_base(samsum_test['dialogue_prompt'][5])[0]["summary_text"])

we're meeting for the drinks in the evening anyway and we'll be going back to the apartment together? Hilary: I'm sooo tired after yesterday .


In [ ]:
print(pipeline_1(samsum_test['dialogue_prompt'][5])[0]["summary_text"])

Benjamin and Elliot will meet at lunchtime and take the keys and take a nap. They'll meet at the entrance to the conference hall at 2 pm and then go to La Cantina.


In [ ]:
print(pipeline_2(samsum_test['dialogue_prompt_new'][5])[0]["summary_text"])

Hilary and Elliot will meet at lunchtime, take the keys and take a nap. They'll meet at La Cantina at 2 pm.


In [ ]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
from transformers import logging
logging.set_verbosity_error()

In [ ]:
summaries_base = []
for out in tqdm(pipeline_base(KeyDataset(samsum_test, "dialogue_prompt"))):
    summaries_base.append(out[0]['summary_text'])

In [ ]:
summaries_1 = []
for out in tqdm(pipeline_1(KeyDataset(samsum_test, "dialogue_prompt"))):
    summaries_1.append(out[0]['summary_text'])

In [ ]:
summaries_2 = []
for out in tqdm(pipeline_2(KeyDataset(samsum_test, "dialogue_prompt_new"))):
    summaries_2.append(out[0]['summary_text'])

In [ ]:
summaries_all = summaries_base
summaries_1 = summaries_base[819:819*2]
summaries_2 = summaries_base[819*2:]
summaries_base = summaries_base[:819]

In [ ]:
import pandas as pd

In [ ]:
res = []
models = ['base', 'fine-tuned', 'fine-tuned new prompt']
preds = [summaries_base, summaries_1, summaries_2]
for model, predictions in zip(models, preds):
    row = {"model": model}
    metrics = rouge.compute(predictions=predictions, references=samsum_test['summary'], use_stemmer=True)
    row.update(metrics)
    res.append(row)
res = pd.DataFrame.from_records(res)
res

,model,rouge1,rouge2,rougeL,rougeLsum
0,base,0.291530,0.084075,0.217757,0.217957
1,fine-tuned,0.423714,0.184818,0.327715,0.327795
2,fine-tuned new prompt,0.432573,0.193276,0.336552,0.336778


### Conclusion

Although adding custom prompt during the fine-tuning breaks consitency with the pre-training stage, fine-tuned model with custom prompt showed better performance in all considered rouge metrics on the test set. 

## Save predictions

In [ ]:
samsum_test = samsum_test.add_column('summaries_base', summaries_base)
samsum_test = samsum_test.add_column('summaries_tuned', summaries_1)
samsum_test = samsum_test.add_column('summaries_tuned_new_prompt', summaries_2)

In [ ]:
samsum_test.to_csv("/kaggle/working/predictions.csv")